In [1]:
import keras
import tensorflow as tf
from keras.datasets import reuters
from keras.preprocessing.text import Tokenizer
from keras.layers import BatchNormalization
from keras.models import Model
from keras.layers import Dense, Conv1D, Flatten, Dropout, Input
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(path="reuters.npz",
                                                         num_words=None,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=113,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)

In [3]:
#Look at a small part of the data
print('Training data: ', x_train[0])
print('Training label: ', y_train[0])
print('Length of training data', len(x_train))
print('Length of test data', len(x_test))

Training data:  [1, 27595, 28842, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]
Training label:  3
Length of training data 8982
Length of test data 2246


In [4]:
word_index = reuters.get_word_index(path="reuters_word_index.json")
#Check to see what index the word sport is, a rudimentary test of the index loading
word_index["sport"]

13074

In [5]:
#The index is organized to look up the integer value, it would be better to look up the key
integer_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Now we can search for the word that aligns to a certain key, we looked up sport before so lets check that index
print('The word at index 13074 is: ',integer_word_index[13074])

#how many different words are in the index
print('There are', len(integer_word_index)+1, 'words in the word index')

The word at index 13074 is:  sport
There are 30980 words in the word index


In [6]:
#Max words in an article
max_words = 10000
#46 labels
LABEL_DIMENSIONS = max(y_train)+1

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

y_train = keras.utils.to_categorical(y_train, LABEL_DIMENSIONS)
y_test = keras.utils.to_categorical(y_test, LABEL_DIMENSIONS)

print(x_train[0])
print(len(x_train[0]))

print(y_train[0])
print(len(y_train[0]))

[0. 1. 0. ... 0. 0. 0.]
10000
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
46


In [32]:
def build_model(keep_prob=0.5, optimizer='adam'):
    inputs = Input(shape=[10000,], name="input") 
    
    #Convolution 1
    conv1 = Dense(128, activation="relu", name="conv_1")(inputs)

    #Convolution 2
    conv2 = Dense(64,  activation="relu", name="conv_2")(conv1)
   
    #output
    prediction=Dense(46, activation="softmax", name ="softmax")(conv2)

    # finalize and compile
    model = Model(inputs=inputs, outputs=prediction)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])
    return model

In [33]:
def create_hyperparameters():
    batches=[10,20,30,40,50,60,70]
    optimizers = ['rmsprop','adam','adadelta']
    dropout=np.linspace(0.1,0.5,5)
    epochs = [5]
    return {"batch_size":batches, "optimizer":optimizers, "keep_prob":dropout, "epochs":epochs,}


In [40]:
model = KerasClassifier(build_fn=build_model, verbose=2)
hyperparameters=create_hyperparameters()

In [41]:
search = RandomizedSearchCV(estimator=model, param_distributions=hyperparameters, n_iter=10, n_jobs=1, cv=3, verbose=2)

In [42]:
search.fit(x_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] optimizer=adadelta, keep_prob=0.4, epochs=5, batch_size=10 ......
Epoch 1/5
 - 6s - loss: 1.3778 - acc: 0.6894
Epoch 2/5
 - 4s - loss: 0.7581 - acc: 0.8282
Epoch 3/5
 - 4s - loss: 0.4742 - acc: 0.8901
Epoch 4/5
 - 4s - loss: 0.3135 - acc: 0.9324
Epoch 5/5
 - 4s - loss: 0.2348 - acc: 0.9484
[CV]  optimizer=adadelta, keep_prob=0.4, epochs=5, batch_size=10, total=  22.1s
[CV] optimizer=adadelta, keep_prob=0.4, epochs=5, batch_size=10 ......


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   23.5s remaining:    0.0s


Epoch 1/5
 - 6s - loss: 1.3834 - acc: 0.6899
Epoch 2/5
 - 4s - loss: 0.7583 - acc: 0.8333
Epoch 3/5
 - 4s - loss: 0.4798 - acc: 0.8993
Epoch 4/5
 - 4s - loss: 0.3273 - acc: 0.9307
Epoch 5/5
 - 4s - loss: 0.2427 - acc: 0.9471
[CV]  optimizer=adadelta, keep_prob=0.4, epochs=5, batch_size=10, total=  22.2s
[CV] optimizer=adadelta, keep_prob=0.4, epochs=5, batch_size=10 ......
Epoch 1/5
 - 6s - loss: 1.3946 - acc: 0.6830
Epoch 2/5
 - 4s - loss: 0.7481 - acc: 0.8267
Epoch 3/5
 - 4s - loss: 0.4786 - acc: 0.8925
Epoch 4/5
 - 4s - loss: 0.3156 - acc: 0.9304
Epoch 5/5
 - 4s - loss: 0.2335 - acc: 0.9471
[CV]  optimizer=adadelta, keep_prob=0.4, epochs=5, batch_size=10, total=  22.3s
[CV] optimizer=rmsprop, keep_prob=0.1, epochs=5, batch_size=50 .......
Epoch 1/5
 - 3s - loss: 1.5078 - acc: 0.6820
Epoch 2/5
 - 1s - loss: 0.7199 - acc: 0.8460
Epoch 3/5
 - 1s - loss: 0.4159 - acc: 0.9137
Epoch 4/5
 - 1s - loss: 0.2671 - acc: 0.9436
Epoch 5/5
 - 1s - loss: 0.1978 - acc: 0.9566
[CV]  optimizer=rmsprop

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  8.3min finished


Epoch 1/5
 - 8s - loss: 1.2426 - acc: 0.7335
Epoch 2/5
 - 5s - loss: 0.7084 - acc: 0.8476
Epoch 3/5
 - 5s - loss: 0.4950 - acc: 0.8946
Epoch 4/5
 - 5s - loss: 0.3774 - acc: 0.9233
Epoch 5/5
 - 5s - loss: 0.3252 - acc: 0.9353


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f52bb67b208>,
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'batch_size': [10, 20, 30, 40, 50, 60, 70], 'optimizer': ['rmsprop', 'adam', 'adadelta'], 'keep_prob': array([0.1, 0.2, 0.3, 0.4, 0.5]), 'epochs': [5]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [43]:
print(search.best_params_)

{'optimizer': 'rmsprop', 'keep_prob': 0.2, 'epochs': 5, 'batch_size': 10}


In [7]:
# create model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(10000, )))
model.add(tf.keras.layers.Dropout(0.8))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(LABEL_DIMENSIONS, activation=tf.nn.softmax))

optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1280128   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 46)                2990      
Total params: 1,291,374
Trainable params: 1,291,374
Non-trainable params: 0
_________________________________________________________________


In [8]:
batch_size = 50
epochs = 10

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2)
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 2s 346us/step - loss: 3.0428 - acc: 0.2788 - val_loss: 1.7043 - val_acc: 0.5899
Epoch 2/10
7185/7185 [==============================] - 2s 290us/step - loss: 1.6582 - acc: 0.5981 - val_loss: 1.3423 - val_acc: 0.7140
Epoch 3/10
7185/7185 [==============================] - 2s 311us/step - loss: 1.4033 - acc: 0.6745 - val_loss: 1.2146 - val_acc: 0.7362
Epoch 4/10
7185/7185 [==============================] - 2s 303us/step - loss: 1.2573 - acc: 0.7105 - val_loss: 1.1408 - val_acc: 0.7535
Epoch 5/10
7185/7185 [==============================] - 2s 310us/step - loss: 1.1636 - acc: 0.7304 - val_loss: 1.1126 - val_acc: 0.7624
Epoch 6/10
7185/7185 [==============================] - 2s 304us/step - loss: 1.1079 - acc: 0.7473 - val_loss: 1.0859 - val_acc: 0.7696
Epoch 7/10
7185/7185 [==============================] - 2s 293us/step - loss: 1.0327 - acc: 0.7621 - val_loss: 1.0576 - val_acc: 0.7824
